# IHLT-MAI S7: Word sense disambiguation

In [21]:
import nltk
from nltk.metrics import jaccard_distance
from scipy.stats import pearsonr
from nltk import pos_tag
#nltk.download('maxent_ne_chunker')
#nltk.download('conll2000')
from nltk import word_tokenize, pos_tag, ne_chunk
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('universal_tagset')

We start by reading the trial set sentences and its respective gold standard.

In [22]:
with open('trial/STS.input.txt') as fp:
    data = fp.readlines()
    
with open('trial/STS.gs.txt') as e:
    gs = e.readlines()

We define a function to retrieve the named entities of a given sentence by using the ne_chunk() function from NTLK. This NLTK function has a parameter select between the binary and non-binary options. According to the NLTK documentation (https://www.nltk.org/book/ch07.html), "if we set the parameter binary=True, then named entities are just tagged as NE; otherwise, the classifier adds category labels such as PERSON, ORGANIZATION, and GPE". GPE stands for Geo-political entity.

With regard to our function, we have implemented different options and policies apart from selecting the binary option for the NLTK function:
- Named entity policy (ne_policy): In the case that a token or a sequence of tokens has an assigned named entity, there are two different options that we have implemented:
    - Replacing the detected tokens by the given entity tag (eg. 'John Smith' -> 'PERSON'): 'ne'.
    - Considering the detected sequence as a single element of the set (eg. instead of adding 'John' and 'Smith' separately, we will add 'John Smith'): 'unified_token'
- No named entity policy (no_ne_policy): In the case that a given token is not assigned a named entity, we provide two options:
    - Leaving the original word.
    - Assigning its respective POS tag instead.

In [23]:
def get_nes(sentence, binary, ne_policy, no_ne_policy):
    assert binary in [True, False]
    assert ne_policy in ['ne', 'unified_token']
    assert no_ne_policy in ['word', 'pos']
    x = pos_tag(word_tokenize(sentence))
    nes = ne_chunk(x, binary=binary)
    res = set([])
    for tree_element in nes:
        if type(tree_element) == nltk.tree.Tree:
            if ne_policy == 'ne':
                res.add(tree_element.label())
            else:
                for element in tree_element:
                    res.add(element[0])
        else:
            if no_ne_policy == 'word':
                res.add(tree_element[0]) # word
            else:
                res.add(tree_element[1]) # pos
    return res

Finally, we define two functions for the two different ways we have investigated to compute the Jaccard distance:
- Synsets (eval_synsets): Compute the Jaccard distance between the set of the disambiguated synsets of the first
    sentence and the set of disambiguated synsets of the second sentence.
- Definitions (eval_definitions): Compute the Jaccard distance between the set of words in the **definitions** of the disambiguated synsets of the first sentence and the ones in the second sentence.

In [24]:
def jaccard_with_nes(sent1, sent2, binary, ne_policy, no_ne_policy):
    nes_sent1 = get_nes(sent1, binary, ne_policy, no_ne_policy)
    nes_sent2 = get_nes(sent2, binary, ne_policy, no_ne_policy)
    return jaccard_distance(nes_sent1, nes_sent2)

jaccard_binary_ne_word = []
jaccard_binary_ne_pos = []
jaccard_binary_unified_token_word = []
jaccard_binary_unified_token_pos = []
jaccard_no_binary_ne_word = []
jaccard_no_binary_ne_pos = []
jaccard_no_binary_unified_token_word = []
jaccard_no_binary_unified_token_pos = []
gold = []
for index, line in enumerate(data):
    (num, sent1, sent2) = line.split('\t')
    jaccard_binary_ne_word.append(jaccard_with_nes(sent1, sent2, binary=True, ne_policy='ne', no_ne_policy='word'))
    jaccard_binary_ne_pos.append(jaccard_with_nes(sent1, sent2, binary=True, ne_policy='ne', no_ne_policy='pos'))
    jaccard_binary_unified_token_word.append(jaccard_with_nes(sent1, sent2, binary=True, ne_policy='unified_token', no_ne_policy='word'))
    jaccard_binary_unified_token_pos.append(jaccard_with_nes(sent1, sent2, binary=True, ne_policy='unified_token', no_ne_policy='pos'))
    jaccard_no_binary_ne_word.append(jaccard_with_nes(sent1, sent2, binary=False, ne_policy='ne', no_ne_policy='word'))
    jaccard_no_binary_ne_pos.append(jaccard_with_nes(sent1, sent2, binary=False, ne_policy='ne', no_ne_policy='pos'))
    jaccard_no_binary_unified_token_word.append(jaccard_with_nes(sent1, sent2, binary=False, ne_policy='unified_token', no_ne_policy='word'))
    jaccard_no_binary_unified_token_pos.append(jaccard_with_nes(sent1, sent2, binary=False, ne_policy='unified_token', no_ne_policy='pos'))
    gold.append(int(gs[index].split('\t')[1][0]))

print('Pearson correlation between gold and jaccard distance using binary NE, NEs and words:', pearsonr(gold, jaccard_binary_ne_word)[0])
print('Pearson correlation between gold and jaccard distance using binary NE, NEs and POS:', pearsonr(gold, jaccard_binary_ne_pos)[0])
print('Pearson correlation between gold and jaccard distance using binary NE, unified tokens and words:', pearsonr(gold, jaccard_binary_unified_token_word)[0])
print('Pearson correlation between gold and jaccard distance using binary NE, unified tokens and POS:', pearsonr(gold, jaccard_binary_unified_token_pos)[0])
print('Pearson correlation between gold and jaccard distance using non-binary NE, NEs and words:', pearsonr(gold, jaccard_no_binary_ne_word)[0])
print('Pearson correlation between gold and jaccard distance using non-binary NE, NEs and POS:', pearsonr(gold, jaccard_no_binary_ne_pos)[0])
print('Pearson correlation between gold and jaccard distance using non-binary NE, unified tokens and words:', pearsonr(gold, jaccard_no_binary_unified_token_word)[0])
print('Pearson correlation between gold and jaccard distance using non-binary NE, unified tokens and POS:', pearsonr(gold, jaccard_no_binary_unified_token_pos)[0])

Pearson correlation between gold and jaccard distance using binary NE, NEs and words: 0.3621104086764575
Pearson correlation between gold and jaccard distance using binary NE, NEs and POS: 0.5320054206469562
Pearson correlation between gold and jaccard distance using binary NE, unified tokens and words: 0.3962389776119233
Pearson correlation between gold and jaccard distance using binary NE, unified tokens and POS: 0.5141109167924274
Pearson correlation between gold and jaccard distance using non-binary NE, NEs and words: 0.3962389776119233
Pearson correlation between gold and jaccard distance using non-binary NE, NEs and POS: 0.5141109167924274
Pearson correlation between gold and jaccard distance using non-binary NE, unified tokens and words: 0.3962389776119233
Pearson correlation between gold and jaccard distance using non-binary NE, unified tokens and POS: 0.5141109167924274


### Comparison and conclusions

In previous sessions, tackling the exact same problem with other approaches, we obtained:
- Session 2 (Document): 0.397, with plain words. Plain words have high morphological variance (eg. a gerund and its correspomding infinitive would be considered as different words).
- Session 3 (Morphology): 0.491, with lemmas. Lemmas improve the result because words are 'normalized' to its 'canonical' form and in word-level settings morphological information (at least in English) introduces "noise". 
- Session 7 (Word Sense Disambiguation): 0.470 and 0.454 by using synsets and words in the definitions of the synsets, respectively. It improved the results with respect to Session 2, but it didn't with respect to Session 3, even though the approach was more complex, and we believe that this is the case because the Lesk algorithm has a relatively low accuracy.

In this session, we are only getting better results than in Session 3 when using POS instead of the words that aren't tagged with named entities. Using POS instead of the words in the case of words that aren't tagged with any named entity consistently outperform the other cases. The reason why this may be happening is that at least with the sentences we have, the most important semantic informatiom is related to the named entities, and using POS for the rest of the words decreases the variance of possible words, but it is still unclear for us and could be investigated further in other assignments (blocks and project).

The difference between using binary or non-binary NE tagging and the difference between using the actual NE tags or the 'unified tokens' do not seem to be that relevant. The best option seems to be binary NE tagging with NEs and POS.

However, all conclusions have to be extracted with caution, since we are only working with the trial set. As a future work, bigger sets should be used, but for now we are focused on the block 2 and the project.